# Generate your predictions on test data and save the results in required format

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# define the ResNet model
class ResNetCIFAR(nn.Module):
    def __init__(self, ):
        super(ResNetCIFAR, self).__init__()

        # First layer of 3x3 conv
        self.conv1 = nn.Conv2d(3, 16, 3, padding = "same")
        
        # Next six layers of 3x3 conv with 32x32 output maps
        self.conv2 = nn.Conv2d(16, 16, 3, padding = "same")
        self.conv3 = nn.Conv2d(16, 16, 3, padding = "same") # shortcut
        self.conv4 = nn.Conv2d(16, 16, 3, padding = "same")
        self.conv5 = nn.Conv2d(16, 16, 3, padding = "same") # shortcut
        self.conv6 = nn.Conv2d(16, 16, 3, padding = "same")
        self.conv7 = nn.Conv2d(16, 16, 3, padding = "same") # shortcut
        
        # Next six layers of 3x3 conv with 16x16 output maps
        self.conv8 = nn.Conv2d(16, 32, 3, padding = (1,1), stride = 2)
        self.conv9 = nn.Conv2d(32, 32, 3, padding = "same") # shortcut
        self.conv10 = nn.Conv2d(32, 32, 3, padding = "same")
        self.conv11 = nn.Conv2d(32, 32, 3, padding = "same") # shortcut
        self.conv12 = nn.Conv2d(32, 32, 3, padding = "same")
        self.conv13 = nn.Conv2d(32, 32, 3, padding = "same") # shortcut
        
        # Next six layers of 3x3 conv with 8x8 output maps
        self.conv14 = nn.Conv2d(32, 64, 3, padding = (1,1), stride = 2)
        self.conv15 = nn.Conv2d(64, 64, 3, padding = "same") # shortcut
        self.conv16 = nn.Conv2d(64, 64, 3, padding = "same")
        self.conv17 = nn.Conv2d(64, 64, 3, padding = "same") # shortcut
        self.conv18 = nn.Conv2d(64, 64, 3, padding = "same")
        self.conv19 = nn.Conv2d(64, 64, 3, padding = "same") # shortcut
        
        # Fully connected layers
        self.fc = nn.Linear(64,10)
        
        # Batch Normalization Layers
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(16)
        self.bn4 = nn.BatchNorm2d(16)
        self.bn5 = nn.BatchNorm2d(16)
        self.bn6 = nn.BatchNorm2d(16)
        self.bn7 = nn.BatchNorm2d(16)
        self.bn8 = nn.BatchNorm2d(32)
        self.bn9 = nn.BatchNorm2d(32)
        self.bn10 = nn.BatchNorm2d(32)
        self.bn11 = nn.BatchNorm2d(32)
        self.bn12 = nn.BatchNorm2d(32)
        self.bn13 = nn.BatchNorm2d(32)
        self.bn14 = nn.BatchNorm2d(64)
        self.bn15 = nn.BatchNorm2d(64)
        self.bn16 = nn.BatchNorm2d(64)
        self.bn17 = nn.BatchNorm2d(64)
        self.bn18 = nn.BatchNorm2d(64)
        self.bn19 = nn.BatchNorm2d(64)

    def forward(self, x):
        # First convolution layer
        out_sc1 = F.relu(self.bn1(self.conv1(x)))
        
        # Next 6 with 32x32 maps, 16 filters
        out = F.relu(self.bn2(self.conv2(out_sc1)))
        out_sc2 = F.relu(self.bn3(self.conv3(out)) + out_sc1)
        out = F.relu(self.bn4(self.conv4(out_sc2)))
        out_sc3 = F.relu(self.bn5(self.conv5(out)) + out_sc2)
        out = F.relu(self.bn6(self.conv6(out_sc3)))
        out_sc4 = F.relu(self.bn7(self.conv7(out)) + out_sc3)
        
        # Next 6 with 16x16 maps, 32 filters
        out = F.relu(self.bn8(self.conv8(out_sc4)))
        out_sc4 = F.avg_pool2d(out_sc4, 2)
        out_sc4 = F.pad(out_sc4, (0, 0, 0, 0, 0, 16))
        out_sc5 = F.relu(self.bn9(self.conv9(out)) + out_sc4)
        out = F.relu(self.bn10(self.conv10(out_sc5)))
        out_sc6 = F.relu(self.bn11(self.conv11(out)) + out_sc5)
        out = F.relu(self.bn12(self.conv12(out_sc6)))
        out_sc7 = F.relu(self.bn13(self.conv13(out)) + out_sc6)
        
        # Next 6 with 8x8 maps, 64 filters
        out = F.relu(self.bn14(self.conv14(out_sc7)))
        out_sc7 = F.avg_pool2d(out_sc7, 2)
        out_sc7 = F.pad(out_sc7, (0, 0, 0, 0, 0, 32))
        out_sc8 = F.relu(self.bn15(self.conv15(out)) + out_sc7)
        out = F.relu(self.bn16(self.conv16(out_sc8)))
        out_sc9 = F.relu(self.bn17(self.conv17(out)) + out_sc8)
        out = F.relu(self.bn18(self.conv18(out_sc9)))
        out = F.relu(self.bn19(self.conv19(out)) + out_sc9)
        
        # Average pooling to get size 64 tensors
        out = F.avg_pool2d(out, 8)
        # Flatten the output tensor
        out = out.view(out.size(0), -1)
        # Fully-Connected Layer
        out = self.fc(out)
        return out

In [4]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from tools.dataset import CIFAR10

DATA_ROOT = "./data"
BATCH_SIZE = 100

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_set = CIFAR10(
    root=DATA_ROOT, 
    mode='test', 
    download=True,
    transform=transform_test
)

# do NOT shuffle your test data loader!!!!!!!!!!!!!!!!
# otherwise the order of samples will be messed up
# and your test accuracy is likely to drop to random guessing level
test_loader = DataLoader(
    test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

#########################################################
# use your model to generate predictions on test data
# and save the results into variable "results"
# "results" should be either a numpy array or a torch tensor with length of 10000

# initialize a resnet and load trained weights
net = ResNetCIFAR()
state_dict = torch.load('./saved_model_resnet/simplenn.pth') # change the path to your own checkpoint file
net.load_state_dict(state_dict['state_dict'])
net.cuda()

# remember to switch to eval mode whenever you are making inference
net.eval()

results = []
with torch.no_grad():
    for x in test_loader:
        results.append(
            net(x.cuda()).argmax(1)
        )

# convert results to numpy array
results = torch.cat(results).cpu().numpy()
assert len(results) == 10000

#########################################################
with open('predictions.csv', 'w') as fp:
    fp.write("Id,Label\n")
    for i in range(len(results)):
        fp.write("%d,%d\n" %(i, results[i]))

Using downloaded and verified file: ./data/cifar10_test_F22.zip
Extracting ./data/cifar10_test_F22.zip to ./data
Files already downloaded and verified
